# GPU Version to run on MARCC

This notebook is assummed to be launched from the top level of the AutomaticLP github repo (where the notebook file is)

This also assummes that you've already cloed the tensorflow object detection project
https://github.com/tensorflow/models (since you need to do this to add the python path to  the bashrc file which has to be done before launching jupyter lab

Run with the LPproject kernel

## Setup / Installations

Add
<code>
export PYTHONPATH=$PYTHONPATH:`pwd`
export PYTHONPATH=$PYTHONPATH:`pwd`/slim

to your ~/.bashrc file where pwd is the full path to the models/research directory

### Tests that Tensorflow GPU is installed & this notebook can see the GPU 

These don't need to be run everytime but are here as a sanity / debugging check

In [1]:
# test that tensorflow can access a GPU
import tensorflow as tf
tf.test.is_built_with_cuda()

True

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7710612676275185137
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8925461647664607157
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11329617920
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9921250646624444714
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:06:00.0, compute capability: 3.7"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12711171375488494862
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
print ('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


Default GPU Device: /device:GPU:0


In [4]:
%%bash 
cd models/research
FILE=protobuf.zip
if [ ! -f "$FILE" ]; then
    wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
    unzip protobuf.zip
fi
./bin/protoc object_detection/protos/*.proto --python_out=.
pip install --user .

Processing /home-net/home-1/cpaolic1@jhu.edu/code/AutomaticLP/models/research
  Stored in directory: /tmp/pip-ephem-wheel-cache-emyhnsgg/wheels/ce/86/0c/1a704f75667a2580a756b306611e0a697afafce50c023988d0
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [5]:
!ml cuda/9.0
!export CUDA_VISIBLE_DEVICES=1
!echo ${CUDA_VISIBLE_DEVICES}

0


## Generating Datasets

The ScriptForBashFile generates both the training and test data and TFRecords from the image files

(For now hard coded to 100 images - TODO: switch back to user input, note then this won't be able to run with a bash script)

In [6]:
%run ScriptForBashFile

TrainingImages_2020-03-22_10-03
60  Training Images Generated
TestImages_2020-03-22_10-03
40  Test Images Generated



## Train

To watch training run <code>tensorboard --logdir=trainingOutput<code> in a terminal instance

In [7]:
# If training output directory dosen't exist create it
!mkdir -p trainingOutput

In [ ]:
%cd models/research
%run object_detection/legacy/train.py --logtostderr --train_dir=../../trainingOutput/ --pipeline_config_path=../../FromScratch/models/model/ssd_mobilenet_v1_coco.config
%cd ../..

/home-net/home-1/cpaolic1@jhu.edu/code/AutomaticLP/models/research
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
Use object_detection/model_main.py.


W0322 10:03:45.953500 47040260826560 deprecation.py:323] From /home-1/cpaolic1@jhu.edu/.conda/envs/tensorflow/lib/python3.7/site-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.


W0322 10:03:45.954335 47040260826560 module_wrapper.py:139] From /home-net/home-1/cpaolic1@jhu.edu/code/AutomaticLP/models/research/object_detection/legacy/train.py:91: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



W0322 10:03:45.959506 47040260826560 module_wrapper.py:139] From /home-net/home-1/cpaolic1@jhu.edu/code/AutomaticLP/models/research/object_detection/legacy/train.py:96: The name tf.gfile.Copy is deprecated. Please use tf.io.gfile.copy instead.



Instructions for updating:
Please switch to tf.train.create_global_step


...

I0322 10:58:09.695632 47040260826560 learning.py:507] global step 6150: loss = 1.3477 (0.518 sec/step)


INFO:tensorflow:global step 6151: loss = 1.0894 (0.541 sec/step)


I0322 10:58:10.238626 47040260826560 learning.py:507] global step 6151: loss = 1.0894 (0.541 sec/step)


INFO:tensorflow:global step 6152: loss = 1.0714 (0.510 sec/step)


I0322 10:58:10.750903 47040260826560 learning.py:507] global step 6152: loss = 1.0714 (0.510 sec/step)
